In [41]:
import datetime
import csv
import json

In [42]:
degradation_coefficient_low = 1.06
degradation_coefficient_high = 1.1
max_score = 3

In [65]:
def get_file(file):
    reader = csv.DictReader(open(file, 'r'))
    data = []
    for row in reader:
        for key, value in row.items():
            if key in ['distance', 'hr', 'elevation']:
                row[key] = float(row[key])
        data.append(row)

    return data

def json_data(file, data):
    with open(file, 'w') as fp:
        json.dump(data, fp)

def multiply_n(number, n):
    result = 1
    for i in range(0, n):
        result *= number

    return result

def time_to_seconds(time):
    result = 0
    time_list = time.split(':')

    for k, item in enumerate(reversed(time_list)):
        result += float(item) * multiply_n(60, k)

    return result

def seconds_to_time(number_of_seconds):
    result = str(datetime.timedelta(seconds=number_of_seconds))
    return result

def get_pace(distance, time):
    pace = (time / distance)
    return pace

def get_time_by_pace(distance, pace):
    time = (distance * pace)
    return time

# ------------------------ [ main functions ] ---------------------------------

def linear(desired_distance, pace):
    #linear time distance km and pace 
    predicted_time = get_time_by_pace(desired_distance, time_to_seconds(pace))

    return predicted_time

def riegel(desired_distance, distance, pace, degradation_coefficient):
    # T2 = T1 * (D2 / D1)^C
    time_seconds = get_time_by_pace(distance, time_to_seconds(pace))
    predicted_time = time_seconds * ( desired_distance / distance ) ** degradation_coefficient

    return predicted_time

def get_coefficient_by_hr(hr):
    return 1

def get_coefficient_by_score(score):
    # 3% should be the highest coefficient
    # 0% should be the lowest coefficient

    # 3 = 100% of the (high-low)
    # N = X%
    # X% = (N / 3) * 100
    coefficient_ratio = score / max_score
    coefficient_increase = (degradation_coefficient_high - degradation_coefficient_low) * coefficient_ratio

    degradation_coefficient = degradation_coefficient_low + coefficient_increase

    return degradation_coefficient

def get_score(distance, elevation):
    # score = elevation(m) / distance(m) * 100
    score = (elevation / (distance * 1000)) * 100

    return score

def calculate(distance, elevation):
    prediction_data = []

    degradation_coefficient = get_coefficient_by_score(get_score(distance, elevation))

    data = get_file('data.csv')

    for num, run in enumerate(data):
        print( "Run log number %s" % (num+1))

        LP = linear(distance, run['pace'])
        PH = riegel(distance, run['distance'], run['gap'], degradation_coefficient_high)
        PM = riegel(distance, run['distance'], run['gap'], degradation_coefficient)
        LG = linear(distance, run['pace'])
        PL = riegel(distance, run['distance'], run['gap'], degradation_coefficient_low)

        prediction_data_value = {
            'LP': {'time': seconds_to_time(LP)},'PH': {'time': seconds_to_time(PH)},'PM': {'time': seconds_to_time(PM)},
            'LG': {'time': seconds_to_time(LG)},'PL': {'time': seconds_to_time(PL)},}
        prediction_data.append(prediction_data_value)

        print ("Top limit","\n","Linear pace prediction: %s" % prediction_data_value['LP']['time'],
              "\n", "Pete Reigel with a high GAP prediction: %s" % prediction_data_value['PH']['time'])
        print ("Middle limit","\n","Pete Reigel with a medium GAP prediction: %s" % 
               prediction_data_value['PM']['time'])
        print ("Low limit","\n","Linear GAP prediction: %s" % prediction_data_value['LG']['time'],"\n",
               "Pete Reigel with a low GAP prediction: %s" % prediction_data_value['PL']['time'],"\n")

        json_data('output.json', prediction_data)
        
#42.195 km  = 26.2 miles (Marathon)
calculate(42.195, 82)

Run log number 1
Top limit 
 Linear pace prediction: 6:04:17.010000 
 Pete Reigel with a high GAP prediction: 4:01:43.156436
Middle limit 
 Pete Reigel with a medium GAP prediction: 3:32:12.857185
Low limit 
 Linear GAP prediction: 6:04:17.010000 
 Pete Reigel with a low GAP prediction: 3:30:18.560837 

Run log number 2
Top limit 
 Linear pace prediction: 5:24:54.090000 
 Pete Reigel with a high GAP prediction: 1:38:25.216916
Middle limit 
 Pete Reigel with a medium GAP prediction: 1:26:02.356598
Low limit 
 Linear GAP prediction: 5:24:54.090000 
 Pete Reigel with a low GAP prediction: 1:25:14.506502 

Run log number 3
Top limit 
 Linear pace prediction: 6:09:54.570000 
 Pete Reigel with a high GAP prediction: 0:03:40.626387
Middle limit 
 Pete Reigel with a medium GAP prediction: 0:03:11.875691
Low limit 
 Linear GAP prediction: 6:09:54.570000 
 Pete Reigel with a low GAP prediction: 0:03:10.028994 

Run log number 4
Top limit 
 Linear pace prediction: 5:50:55.305000 
 Pete Reigel wit